# Assignment 3
## Neural Style Transfer


The seminal work of Gatys et al. demonstrated the power of Convolutional Neural Networks (CNN) in creating artistic imagery by separating and recombining image content and style. This process of using CNN to render a content image in different styles is referred to as Neural Style Transfer (NST). Since then, NST has become a trending topic both in academic literature and industrial applications.

The objective of this assigment is to understand NST. To this end, the specific points for this assignment are:

+ Read (carefully) this paper: (https://arxiv.org/abs/1705.04058)
+ Look at this implementation: https://github.com/makkam012/keras_style_transfer/blob/master/keras_style_transfer.py
+ Re-code the previous implementation (in tf or keras) in order to have a clear implementation that can be used for pedagogical purposes. For example, use more data visualization, simplify the code, etc.
+ Write a tutorial notebook (include latex formulas, figures, etc.) with your code about NST.

*Deliverable*: Notebook.

*Submission date*: 1/6/2018.

*Scoring criteria*: the most important feature for getting a good scoring of this assignment will be the pedagogical value of your notebook.

### Neural Style Transfer (NST)
- Neural Style Transfer (NST) is a field at the intersection between Deep Learning, Computer vision, and Art. The Aim of NST is to render images in direrent styles by means of Convolutional Neural Networks (CNNs).

- The first work in NST, proposed by Gatys et al. in $"A\ Neural\ Algorithm\ of\ Artistic\ Style"$, s to combine natural images with famous styles. The results can be seen in the figure below.

<img src="./images/nst.png">

- The main idea of their algorithm is to optimize an image, with the aim of matching desired feature distributions, combining the **content** of the original image with the **style** of the artwork. The content of the original image is modeled with the feature activations of a pretrained CNN, whereas the style of the artwork is modeled with summary feature statistics.

In [3]:
import time

import numpy as np
import tensorflow.contrib.keras.api.keras.backend as K
from scipy.misc import imsave, imresize
#import skimage
from scipy.optimize import fmin_l_bfgs_b
from tensorflow.contrib.keras.api.keras.applications import vgg19
from tensorflow.contrib.keras.api.keras.preprocessing.image import load_img, img_to_array

# Define constants
CONTENT_IMG_PATH = './images/content_1.jpeg'
STYLE_IMG_PATH = './images/style_2.jpg'
OUTPUT_PATH = './output/gen_img_1.png'
# Number of iterations to run
ITER = 10

# Weights of losses
alpha = 0.5 # content weight #0.01
beta = 0.5  # style weight #1.0
TV_WEIGHT = 1.0

# Define the shape of the output image
img = load_img(CONTENT_IMG_PATH)
#img_resized = img.resize((128, 128))

h, w = img.size
img_h = 128
img_w = 128

print("Content image (h, w): (%d,%d)" % (h, w))
print("Output image (h, w): (%d,%d)" % (img_h, img_w))

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Content image (h, w): (825,542)
Output image (h, w): (128,128)


In [4]:
def preprocess(img_path):
    """
    open, resize and format pictures into appropriate tensors of size (img_h, img_w, 3)
    """
    img = load_img(img_path)
    img = img_to_array(img)
    # This dimensions are for Tensorflow backend
    img = imresize(img, (img_h, img_w, 3))
    img = img.astype('float64')
    # Add the batch dimension
    img = np.expand_dims(img, axis=0)
    # Subtracts the mean RGB value of the ImageNet dataset, and transposes RGB to BGR. 
    # this operation is undone in deprocess_image()
    img = vgg19.preprocess_input(img)
    return img


def deprocess_image(img):
    """
    convert a tensor into a valid image
    """
    if K.image_data_format() == 'channels_first':
        print("channel first branch")
        # For Theano
        img = img.reshape((3, img_h, img_w))
        img = img.transpose((1, 2, 0))
    else:
        img = img.reshape((img_h, img_w, 3))
    # Undo vgg19.preprocess_input(img)
    # Remove zero-center by mean pixel
    img[:, :, 0] += 103.939
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    img = img[:, :, ::-1]
    img = np.clip(img, 0, 255).astype('uint8')
    return img

In [5]:
# Create Keras variables. Content and style images as tensors
content_img = K.variable(preprocess(CONTENT_IMG_PATH))
style_img = K.variable(preprocess(STYLE_IMG_PATH))

# define placeholder for the generated image
if K.image_data_format() == 'channels_first':
    print("channel first again")
    gen_img = K.placeholder(shape=(1, 3, img_h, img_w))
else:
    gen_img = K.placeholder(shape=(1, img_h, img_w, 3))

# Create a single tensor containing all three images
input_tensor = K.concatenate([content_img, style_img, gen_img], axis=0)

# Create a vgg19 model by running the input tensor though the vgg19 convolutional
# neural network, excluding the fully connected layers
model = vgg19.VGG19(include_top=False, weights='imagenet', input_tensor=input_tensor)
print('Model loaded')

# Create an output dictionary
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


Model loaded


### Parametric texture modeling with summary statistics

The textures are modeled with second order statistics using the Gram matrix, which represents the correlations between the activations of a convolutional layer.

Let:

- $I_s$: **style** image.
- $I_c$: **content** image.
- $F^l(I_s) \in R^{C \times H \times W}$: feature map  of a sample texture image at layer $l$ of the CNN
- $F^l(I_s)' \in R^{C \times (HW)}$: reshaped version of feature map $F^l(I_s)$

The Gram matrix $G(F^l(I_s)') \in R^{C \times C}$ is computed as follows:

$$ G(F^l(I_s)')  = F^l(I_s)' [ F^l(I_s)' ]^T $$

The following code chunk provides an implementation.

In [6]:
def gram_matrix(x):
    assert K.ndim(x) == 3
    if K.image_data_format() == 'channels_first':
        features = K.flatten(x)
    else:
        features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    # Dot product of the flattened feature map and the transpose of the
    # flattened feature map
    gram = K.dot(features, K.transpose(features))
    return gram

### Objective of the optimization
Given a content image $I_c$ and a style image $I_s$, the aim of the optimization is to find a stylised image that minimizes the following loss function:

$$ L_{total} = \alpha \times L_{content}(I_c, I) + \beta \times L_{style}(I_s, I) $$

--------------------------------------------------

It can be seen that the objective is a linear combination of two components, weighted by scalar $\alpha$ and $\beta$:

- $L_{content}$ : Content loss. Compares the feature maps of the content image $I_c$ and the actual solution (the stylised image $I$)

$$ L_{content} =  \sum_{l \in \{l_c\}} || F^l(I_c) - F^l(I) ||^ 2$$

- $L_{style}$ : Style loss. Compares the Gram-based style representations of the style image $I_s$ and the stylised image $I$

$$ L_{style} = \sum_{l \in \{ l_s \} } || G(F^l(I_s))' - G(F^l(I_s)) ||^2 $$

Where $ \{l_s\} $ and $\{l_c\}$ are the sets of CNN layers used to compute respectively the style and content loss. 

--------------------------------------------------------------------------------------

- $ \{l_s\} $ is composed by the lower layers, since the style is determined by low-level feaures (e.g. color)
- $\{l_c\}$ is composed by the deeper layers, which encode higher level features and semantic content


In [7]:
def style_loss(style, gen):
    assert K.ndim(style) == K.ndim(gen) == 3
    #assert K.ndim(gen) == 3
    S = gram_matrix(style)
    G = gram_matrix(gen)
    #channels = 3
    # size = img_h * img_w
    # Squared Euclidean distance between the gram matrices
    return K.sum(K.square(S - G)) # / (4. * (channels ** 2) * (size ** 2))


def content_loss(content, gen):
    assert K.ndim(content) == K.ndim(gen) == 3
    # assert K.ndim(gen) == 3
    # Squared Euclidean distance between the content image and the actual stylised image
    return K.sum(K.square(gen - content))


def total_variation_loss(x):
    assert K.ndim(x) == 4
    if K.image_data_format() == 'channels_first':
        a = K.square(x[:, :, :img_h - 1, :img_w - 1] - x[:, :, 1:, :img_w - 1])
        b = K.square(x[:, :, :img_h - 1, :img_w - 1] - x[:, :, :img_h - 1, 1:])
    else:
        # Move the image pixel by pixel, and calculate the variance
        a = K.square(x[:, :img_h - 1, :img_w - 1, :] - x[:, 1:, :img_w - 1, :])
        b = K.square(x[:, :img_h - 1, :img_w - 1, :] - x[:, :img_h - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [10]:
loss = K.variable(0.)
# For content loss, we're using the 2nd convolutional layer form the 5th block
layer_features = outputs_dict['block5_conv2']
content_img_features = layer_features[0, :, :, :]
gen_img_features = layer_features[2, :, :, :]

# add content loss
loss = alpha * content_loss(content_img_features, gen_img_features)

feature_layer_names = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
for name in feature_layer_names:
    layer_features = outputs_dict[name]
    style_features = layer_features[1, :, :, :]
    gen_img_features = layer_features[2, :, :, :]

    # add style loss
    # We need to devide the loss by the number of layers that we take into account
    loss += (beta / len(feature_layer_names)) * style_loss(style_features, gen_img_features)
    
loss += TV_WEIGHT * total_variation_loss(gen_img)

# Calculate gradients
grads = K.gradients(loss, gen_img)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

# Define a Keras function
f_output = K.function([gen_img], outputs)

In [11]:
def eval_loss_and_grads(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((1, 3, img_h, img_w))
    else:
        x = x.reshape((1, img_h, img_w, 3))
    # Update the loss and the gradients
    outs = f_output([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values


class Evaluator(object):
    """
    This Evaluator class makes it possible
    to compute loss and gradients in one pass
    while retrieving them via two separate functions,
    "loss" and "grads". This is done because scipy.optimize
    requires separate functions for loss and gradients,
    but computing them separately would be inefficient.
    """
    def __init__(self):
        self.loss_value = None
        self.grads_value = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_value = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grads_value = grad_value
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grads_values = np.copy(self.grads_value)
        self.loss_value = None
        self.grads_value = None
        return grads_values


evaluator = Evaluator()

# Run L-BFGS optimizer
x = preprocess(CONTENT_IMG_PATH)
losses = []

for i in range(ITER):
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(func=evaluator.loss,    # function to minimize
                                     x0=x.flatten(),         # initial guess 
                                     fprime=evaluator.grads, # function gradient
                                     maxiter=300)
    
    print('Step {} - loss: {}'.format(i, min_val))
    # save losses
    losses.append(min_val)
    # Save img
    img = deprocess_image(x)
    imsave('./output/img{}.png'.format(i), img)
    
print('     Image saved. Time: {}'.format(time.time() - start_time))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


Step 0 - loss: 1.151118924012585e+18


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:62: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


KeyboardInterrupt: 